# Environment Setup

For request of AWS instance, see:    
https://github.com/hrzhao76/aws-fpga/blob/master/AWS%20Instructions.ipynb


Notice:
New version(with tensorflow) of Xilinx ML suite has been released. You need to download it first, and then load the image via docker.

```bash 
docker load < xilinx-ml-suite-ubuntu-16.04-xrt-2018.2-tensorflow-1.12.0-mls-1.5.tar.gz
```
You might need to tag the image after loading. type `docker image ls` to see if the new image has tag and name. Change its REPOSITORY to `xilinx-ml-suite-ubuntu-16.04-xrt-2018.2-tensorflow-1.12.0-mls-1.5`.


edit docker_run.sh,

```
#!/usr/bin/env bash

############################################################################
###############################  Hack For AWS ##############################
############################################################################

sudo /opt/xilinx/xrt/bin/awssak query # Need to run this before changing permissions

setperm () {
  sudo chmod g=u $1
  sudo chmod a=u $1
}
setfpgaperm () {
  for f in $1/*; do setperm $f; done
}
for d in /sys/bus/pci/devices/*; do cat $d/class| grep -q "0x058000" && setfpgaperm $d;  done
setperm /sys/bus/pci/rescan

####################################################################################
####################################################################################
####################################################################################

HERE=`dirname $(readlink -f $0)`

mkdir -p $HERE/share
chmod -R a+rwx $HERE/share

xclmgmt_driver="$(find /dev -name xclmgmt\*)"
docker_devices=""
echo "Found xclmgmt driver(s) at ${xclmgmt_driver}"
for i in ${xclmgmt_driver} ;
do
  docker_devices+="--device=$i "
done

render_driver="$(find /dev/dri -name renderD\*)"
echo "Found render driver(s) at ${render_driver}"
for i in ${render_driver} ;
do
  docker_devices+="--device=$i "
done

#sudo \
docker run \
  --rm \
  --net=host \
  --privileged=true \
  --log-driver none \
  -it \
  $docker_devices \
  -v $HERE/share:/opt/ml-suite/share \
  -v /opt/xilinx:/opt/xilinx \
  -w /opt/ml-suite \
  xilinx-ml-suite-ubuntu-16.04-xrt-2018.2-tensorflow-1.12.0-mls-1.5
  bash
```

run `./docker_run.sh`

# tensorflow inference

Official example:     
https://github.com/Xilinx/ml-suite/tree/master/examples/tensorflow

The official example is using inception_v1, if you want to test ResNet_50, after the "One time setup", use the command below: 

1. One time setup
2. Quantize for inference

``` bash
python run.py --quantize --model /opt/models/tensorflow/resnet50_baseline.pb --output_dir work --input_nodes data --output_nodes prob --input_shapes ?,224,224,3
```

3. Partition, Compile, and Run Inference
``` bash
python run.py --validate --model /opt/models/tensorflow/resnet50_baseline.pb --output_dir work --input_nodes data --output_nodes prob --c_input_nodes data --c_output_nodes prob --input_shapes ?,224,224,3

```

Result is like this:
``` bash 

top1_acc:0.72, top5_acc:0.89

```